##### Do not forget to install ffmpeg inorder to use pydub
brew install ffmpeg


In [25]:
from pydub import AudioSegment
import os
import shutil
import math
from deepgram import DeepgramClient, PrerecordedOptions
import os
import markdown
import pdfkit
import os
from groq import Groq

client = Groq(
    api_key='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'
)

# The API key we created in step 3
DEEPGRAM_API_KEY = '82578abdf9477547c395bd610e8109a9700789bc'

def reset_save_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

def audio_spilter(audio):
    directory = 'audio_directory'
    reset_save_directory(directory)
    song = AudioSegment.from_mp3(audio)
    ten_minutes = 10 * 60 * 1000
    audio_length = len(song) / 60000
    print(f'Audio length: {math.floor(audio_length)} minutes')      
    count = 0
    file_paths = []
    for x in range(0, len(song), ten_minutes):
        count += 1
        audio_segment = song[x:x+ten_minutes]
        file_path = os.path.join(os.path.abspath(directory), f"audio_split_{count}.mp3")
        audio_segment.export(file_path, format="mp3")
        file_paths.append(file_path)

    print('Number of splits:', count)
    return file_paths

def speech_to_text(audio_path):
    paths = audio_spilter(audio_path)
    transcript = None
    
    deepgram = DeepgramClient(DEEPGRAM_API_KEY)
    for x in paths:
        print(x)
        with open(f'{x}', mode='rb') as buffer_data:
            payload = { 'buffer': buffer_data }

            options = PrerecordedOptions(
                smart_format=True, model="nova-2", language="en-GB"
            )

            response = deepgram.listen.rest.v('1').transcribe_file(payload, options)
            if transcript:
                print(len(transcript.split(' ')))
                transcript = transcript + response.results.channels[0].alternatives[0].transcript
            else:
                transcript = response.results.channels[0].alternatives[0].transcript

    return transcript

def notes_gen(text):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                    "role": "system",
                    "content": "You are an expert academic writer tasked with producing highly detailed and structured notes based on a provided text. From the given text, extract the key concepts, topics, and themes, and then elaborate on each topic comprehensively. Structure the notes using clear headings, subheadings, and bullet points for clarity. Ensure that the notes are thorough, insightful, and enriched with additional relevant knowledge beyond the text, showcasing deep understanding. The final output should be written in markdown format and consist of 3000 words, with all explanations logically organized and expanded, going beyond the provided content. Focus on delivering advanced, informative, and well-articulated explanations that stand on their own.",

                "role": "user",
                "content": text,
            }
        ],
        model="mixtral-8x7b-32768",
    )

    return chat_completion.choices[0].message.content

def pdf_gen(audio_path):
    text = speech_to_text(audio_path)
    notes = notes_gen(text)
    html_text = markdown.markdown(notes)
    pdfkit.from_string(html_text, "lstm_notes.pdf")
    print("PDF created successfully!")
    


In [26]:
pdf_gen('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/LSTM_audio.mp3')

Audio length: 31 minutes
Number of splits: 4
/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_1.mp3
/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_2.mp3
2059
/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_3.mp3
4060
/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_4.mp3
5915


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01hw8bjjp5fz4v3pv19qrbxkzt` on tokens per minute (TPM): Limit 5000, Used 0, Requested 7721. Please try again in 32.652s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [7]:
text = sppech_to_text('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/LSTM_audio.mp3')

Audio length: 31 minutes
Number of splits: 4
/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_1.mp3
/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_2.mp3
2059
/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_3.mp3
4058
/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_4.mp3
5913


In [8]:
text

"Fine. So, with that motivation let us go to the next module, where we will talk about long short term memory and gated recurrent units okay. So, now, all this is fine in terms of okay, I gave you a derivation on the board and say that this is not required, but can I give you a concrete example where RNNs also need to selectively read write and forget right? Only then you will be convinced that this kind of morphing is bad in the case of RNNs. So, I will start with that example. And then once we agree that we need selective read write and forget, how do we convert this into some mathematical equations, right? Because conceptually it is fine, but you have to write some equations so that the R n can do some computations where you have selective read write and forget, right? So, that is what we are going to do over the rest of the lecture. So, first let me start with a concrete example, where you want to predict the sentiment of a review using an RNN. So, this is the RNN structure, we hav

In [14]:
import os
from groq import Groq

client = Groq(
    api_key='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'
)
def notes_gen(text):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                    "role": "system",
                    "content": "From the given text identify the key topics and concepts and explain them in detail . Organize the notes in a logical flow that reflects the structure of the lecture. Use bullet points, headings, and subheadings where appropriate. The final notes should be in markdown format, please do use headings, subheadings and bullets. Please generate well detailed notes in 3000 words",

                "role": "user",
                "content": text,
            }
        ],
        model="gemma-7b-it",
    )

    return chat_completion.choices[0].message.content

In [15]:
import markdown
import pdfkit

notes = notes_gen(text)
html_text = markdown.markdown(notes)
    
# Save HTML to PDF
pdfkit.from_string(html_text, "lstm_notes.pdf")

print("PDF created successfully!")

PDF created successfully!


In [3]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq(api_key='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')

# Specify the path to the audio file
filename = '/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_1.mp3'# Replace with your audio file!

# Open the audio file
with open(filename, "rb") as file:
    # Create a transcription of the audio file
    transcription = client.audio.transcriptions.create(
      file=(filename, file.read()), # Required audio file
      model="distil-whisper-large-v3-en", # Required model to use for transcription
      prompt="Specify context or spelling",  # Optional
      response_format="json",
      language="en",
      temperature=0.0 
    )
    print(transcription.text)

 So, so with that motivation let's go to the next module where we'll talk about long, short term memory and gated recurrent units. So now all this was fine in terms of okay, I gave you a derivation on the board and say that this is not required but can I give you a concrete example where RNNs also need to selectively read, right and forget, only then you will be convinced that this kind of morphing is bad in the case of Iran. So, I'll start with that example. And then once we agree that we need selective read, write and forget, how do we convert this into some mathematical equations, right? Because conceptually it's fine, but you have to write some equations so that the RN can do some computations where you have selective read, right and forget. So, that's what we are going to do over the rest of the lecture. So, first let me start with a concrete example where you want to predict. the sentiment of a review using an RNN. So, this is the RNN structure. We have done this in the past that

In [28]:
from deepgram import DeepgramClient, PrerecordedOptions
import os

# The API key we created in step 3
DEEPGRAM_API_KEY = '82578abdf9477547c395bd610e8109a9700789bc'

def sppech_tp_text(directory):
    paths = os.listdir(directory)
    transcript = None
    deepgram = DeepgramClient(DEEPGRAM_API_KEY)
    for x in paths:
        with open(f'{directory}/{x}', 'rb') as buffer_data:
            payload = { 'buffer': buffer_data }

            options = PrerecordedOptions(
                smart_format=True, model="nova-2", language="en-GB"
            )

            response = deepgram.listen.rest.v('1').transcribe_file(payload, options)
            if transcript:
                print(len(transcript.split(' ')))
                transcript = transcript + response.results.channels[0].alternatives[0].transcript
            else:
                transcript = response.results.channels[0].alternatives[0].transcript

    return transcript




In [29]:
sppech_tp_text('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory')

2059
4059
5914


"Fine. So, with that motivation let us go to the next module, where we will talk about long short term memory and gated recurrent units okay. So, now, all this is fine in terms of okay, I gave you a derivation on the board and say that this is not required, but can I give you a concrete example where RNNs also need to selectively read write and forget right? Only then you will be convinced that this kind of morphing is bad in the case of RNNs. So, I will start with that example. And then once we agree that we need selective read write and forget, how do we convert this into some mathematical equations, right? Because conceptually it is fine, but you have to write some equations so that the R n can do some computations where you have selective read write and forget, right? So, that is what we are going to do over the rest of the lecture. So, first let me start with a concrete example, where you want to predict the sentiment of a review using an RNN. So, this is the RNN structure, we hav

In [19]:
response.results.channels[0].alternatives[0].transcript

"Fine. So, with that motivation let us go to the next module, where we will talk about long short term memory and gated recurrent units okay. So, now, all this is fine in terms of okay, I gave you a derivation on the board and say that this is not required, but can I give you a concrete example where RNNs also need to selectively read write and forget right? Only then you will be convinced that this kind of morphing is bad in the case of RNNs. So, I will start with that example. And then once we agree that we need selective read write and forget, how do we convert this into some mathematical equations, right? Because conceptually it is fine, but you have to write some equations so that the R n can do some computations where you have selective read write and forget, right? So, that is what we are going to do over the rest of the lecture. So, first let me start with a concrete example, where you want to predict the sentiment of a review using an RNN. So, this is the RNN structure, we hav

In [19]:
from openai import OpenAI
client = OpenAI()

audio_file= open("/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_1.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
resul = model.transcribe('/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_2.mp3')x

In [8]:
import whisper
import torch

# Check if CUDA or MPS (Apple Silicon GPU) is available, otherwise fallback to CPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA (NVIDIA GPU)")
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS (Apple Silicon GPU)")
else:
    device = torch.device('cpu')
    print("Using CPU")

model = whisper.load_model("small")  # Whisper models are loaded on the appropriate device internally

# Transcribe the audio file
result = model.transcribe("/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_1.mp3")

# Print the transcription
print(result["text"])


Using MPS (Apple Silicon GPU)


/Users/naveenpoliasetty/Downloads/David2.0/david2.0/facerecog/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Fine. So, with that motivation let us go to the next module where we will talk about a long short term memory and gated recurrent units ok. So, now, all this was fine in terms of ok I gave you a derivation on the board and say that this is not required, but can I give you a concrete example where RNNs also need to selectively read write and forget right only then you will be convinced that this kind of morphing is bad in the case of RNN. So, I will start with that example and then once we agree that we need selective read write and forget how do we convert this into some mathematical equations right because conceptually it is fine, but you have to write some equations so that the RNN can do some computations where you have selective read write and forget right. So, that is what we are going to do over the rest of the lecture. So, first let me start with a concrete example where you want to predict the sentiment of a review using an RNN. So, this is the RNN structure we have done this 

### We might try these for future

In [ ]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq(api_key='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')

# Specify the path to the audio file
filename = '/Users/naveenpoliasetty/Downloads/David2.0/david2.0/audio_directory/audio_split_1.mp3'# Replace with your audio file!

# Open the audio file
with open(filename, "rb") as file:
    # Create a transcription of the audio file
    transcription = client.audio.transcriptions.create(
      file=(filename, file.read()), # Required audio file
      model="distil-whisper-large-v3-en", # Required model to use for transcription
      prompt="Specify context or spelling",  # Optional
      response_format="json",  # Optional
      language="en",  # Optional
      temperature=0.0  # Optional
    )
    # Print the transcription text
    print(transcription.text)